<a href="https://colab.research.google.com/github/ccarpenterg/introNLP/blob/master/03b_NLP_and_recurrent_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-trained Word Embeddings and RNNs

In [0]:
import torch
from torchtext import data

SEED = 3773

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

In [2]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:07<00:00, 11.6MB/s]


In [3]:
print("Number of training examples: {}".format(len(train_data)))
print("Number of testing examples: {}".format(len(test_data)))

Number of training examples: 25000
Number of testing examples: 25000


In [4]:
print(vars(train_data.examples[0]))

{'text': ['Diagnosis', 'Murder', 'has', 'been', 'shown', 'on', 'most', 'Weekday', 'afternoons', 'on', 'BBC1', 'since', 'I', 'used', 'to', 'watch', 'it', 'while', 'ill', 'from', 'School', 'a', 'good', '10', 'years', 'ago', '-', 'I', 'know', 'I', 'should', "n't", 'really', 'enjoy', 'it', ',', 'in', 'the', 'same', 'way', 'I', 'should', "n't", 'enjoy', "'", 'Murder', 'she', 'Wrote', "'", 'but', 'I', "'m", 'totally', 'addicted', 'to', 'both', 'and', 'even', 'have', 'the', 'DVD', 'box', '-', 'sets', '....', 'OK', 'I', 'know', 'that', "'s", 'sad!<br', '/><br', '/>Dick', 'Van', 'Dyke', 'carries', 'the', 'show', 'as', 'he', 'stars', 'as', 'Dr', '.', 'Mark', 'Sloan', 'a', 'Doctor', 'at', 'Community', 'General', 'Hospital', 'in', 'L.A', 'who', 'is', 'also', 'a', 'Police', 'consultant', 'for', 'the', 'L.A.P.D.', '-', 'his', 'son', 'Steve', '(', 'Barry', 'van', 'Dyke', '-', 'Dick', "'s", 'real', 'life', 'son', ')', 'is', 'a', 'Police', 'Officer', ',', 'who', 'needs', 'his', 'father', "'s", 'help', 

In [5]:
import random

train_data, valid_data = train_data.split(
    random_state=random.seed(SEED),
    split_ratio=0.8
)

print("Number of training examples: {}".format(len(train_data)))
print("Number of validation examples: {}".format(len(valid_data)))
print("Number of testing examples: {}".format(len(test_data)))

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [0]:
MAX_VOCAB_SIZE = 8185

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [7]:
print("Unique tokens in TEXT vocabulary: {}".format(len(TEXT.vocab)))
print("Unique tokens in LABEL vocabulary: {}".format(len(LABEL.vocab)))

Unique tokens in TEXT vocabulary: 8187
Unique tokens in LABEL vocabulary: 2


In [8]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 232678), (',', 220840), ('.', 188920), ('and', 125362), ('a', 125266), ('of', 115884), ('to', 107654), ('is', 87196), ('in', 70206), ('I', 62349), ('it', 61298), ('that', 56438), ('"', 50419), ("'s", 49667), ('this', 48419), ('-', 41945), ('/><br', 41022), ('was', 40196), ('as', 35006), ('with', 34063)]


In [9]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [10]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f51c731e378>, {'pos': 0, 'neg': 1})


In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device
)

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim,
                 hidden_dim, output_dim,
                 n_layers, bidirectional):
        
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.rnn = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, sequences):

        # sequences: (max sequences length, batch size)
        seq_embeddings = self.embedding(sequences)

        # seq_embeddings: (max sequences length, batch size, embedding dim)
        seq_hidden, (hidden, cell) = self.rnn(seq_embeddings)

        # hidden: (num_layers * num_directions, batch size, hidden dim)
        hidden_concat = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)

        # hidden_concat: (batch size, num_directions * hidden dim)
        output = self.fc(hidden_concat)

        # output: (batch size, 1) -> vector
        return output

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 1
BIDIRECTIONAL = True

model = RNN(INPUT_DIM,
            EMBEDDING_DIM,
            HIDDEN_DIM,
            OUTPUT_DIM,
            N_LAYERS,
            BIDIRECTIONAL)

In [18]:
iterator = iter(train_iterator)
batch = next(iterator)
print(batch.text.shape)
print(batch.text[:,0].shape)

torch.Size([592, 64])
torch.Size([592])


In [19]:
model = model.to(device)

output = model(batch.text)

print(output.shape)

torch.Size([64, 1])
